In [1]:
import pandas as pd
import numpy as np

In [2]:
# Чтение датасетов
key = pd.read_csv('hackaton/key.csv')
sample_submission = pd.read_csv('hackaton/sampleSubmission.csv')
store_city_coordinates = pd.read_csv('hackaton/store_city_coordinates.csv')
train = pd.read_csv('hackaton/train.csv')
weather = pd.read_csv('hackaton/weather.csv')
weather_by_city = pd.read_csv('hackaton/weather_by_city.csv')

In [3]:
# Поиск соотношения город - станция
#__________________________________

weather_copy = weather[['station_nbr', 'date', 'tavg', 'preciptotal']].copy()

# Удаление года из дат
weather_by_city.date = weather_by_city.date.apply(lambda x: x[5:])
weather_copy.date = weather_copy.date.apply(lambda x: x[5:])

# Очистка данных
weather_copy.preciptotal = weather_copy.preciptotal.replace('  T', '0.001') # Замена Trace на очень маленькое значение
weather_copy = weather_copy.replace('M', np.nan)
weather_copy = weather_copy.dropna()

weather_copy[['tavg', 'preciptotal']] = weather_copy[['tavg', 'preciptotal']].astype('float32')

# Убираем даты, которых нет в weather_by_city
dates = weather_by_city.date.unique()

def date_in_weather_by_city(date):
    if date not in dates:
        return True
    return False

weather_copy = weather_copy.drop(weather_copy[weather_copy.date.apply(date_in_weather_by_city)].index)

# Группировка данных по станции и дате
weather_copy_grouped = weather_copy.groupby(['station_nbr', 'date']).mean()

# Добавление отсутствующей даты
mean_tavg = (weather_copy_grouped.loc[(8, '01-07')]['tavg'] + weather_copy_grouped.loc[(8, '01-08')]['tavg']) / 2
mean_perc = (weather_copy_grouped.loc[(8, '01-07')]['preciptotal'] + weather_copy_grouped.loc[(8, '01-08')]['preciptotal']) / 2
weather_copy_grouped.loc[(8, '01-06'), 'tavg'] = mean_tavg
weather_copy_grouped.loc[(8, '01-06'), 'preciptotal'] = mean_perc

# Переход в формат "станция/город - средняя сумма температуры и влажности по дате
weather_copy = weather_copy_grouped.reset_index().drop('date', axis=1).groupby('station_nbr').sum()
weather_copy['total'] = weather_copy[['tavg', 'preciptotal']].sum(axis=1)
weather_copy = weather_copy.drop(['tavg', 'preciptotal'], axis=1)

weather_by_city_grouped = weather_by_city.drop('date', axis=1).groupby('city').sum()
weather_by_city_grouped['total'] = weather_by_city_grouped[['avg_temp_c', 'precipitation_mm']].sum(axis=1)
weather_by_city_grouped = weather_by_city_grouped.drop(['avg_temp_c', 'precipitation_mm'], axis=1)

sorted_wbcg = weather_by_city_grouped.sort_values('total')
sorted_wc = weather_copy.sort_values('total')

# Соотношение город - станция
df_city_station_keys = pd.DataFrame({'city': list(sorted_wbcg.index), 'station_nbr': [5] + list(sorted_wc.index)})
df_city_station_keys.to_csv('city_station.csv') # Сохранение датасета

In [4]:
df_city_station_keys

,city,station_nbr
0,Washington,5
1,Columbus,9
2,Seattle,15
3,San Jose,19
4,Los Angeles,2
5,New York,16
6,San Antonio,1
7,Houston,13
8,Chicago,3
9,San Diego,17


In [5]:
# Подготовка данных
#__________________

# Очистка данных
weather_c = weather.replace('M', np.nan)
weather_c = weather_c.dropna()

# Датасет город-станция-магазин
full_keys = key.merge(df_city_station_keys, on=['station_nbr'])

# Добавление названия города
weather_c = weather_c.merge(df_city_station_keys, on=['station_nbr']).drop(['station_nbr'], axis=1)

# Изменение типов данных
weather_c[['tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'resultdir']] = \
    weather_c[['tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', \
               'resultdir']].astype('int')

weather_c = weather_c.replace('  T', '0.001') # Замена Trace на маленькое 0.001
weather_c[['snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'avgspeed']] = \
    weather_c[['snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'avgspeed']].astype('float')

weather_c['city'] = weather_c['city'].astype('category')

codesum_cat = ['+FC', 'FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG+', 'FG', 'BR', 'UP', \
               'HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', \
                'BC', 'BL', 'VC'] # Вариации кодов

for code in codesum_cat:
    weather_c[code] = [False for _ in range(weather_c.shape[0])] # One-Hot-Encoding

# Заполнение Ohe
for i in range(weather_c.shape[0]):
    codes = weather_c.loc[i, 'codesum'].split()
    for code in codes:
        if len(code) > 3:
            weather_c.loc[i, code[:2]] = True
            weather_c.loc[i, code[2:]] = True
        else:
            weather_c.loc[i, code] = True

# Добавление названия города в train
train = train.merge(full_keys[['city', 'store_nbr']], on='store_nbr')

# Добавление товаров и их количества
weather_c = weather_c.merge(train[['city', 'date', 'units', 'item_nbr']], on=['city', 'date'])
weather_c = weather_c.drop(['date', 'codesum'] , axis=1)
weather_c.to_csv('weather_clean.csv')

In [6]:
pd.options.display.max_columns = None
weather_c

,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,city,+FC,FC,TS,GR,RA,DZ,SN,SG,GS,PL,IC,FG+,FG,BR,UP,HZ,FU,VA,DU,DS,PO,SA,SS,PY,SQ,DR,SH,FZ,MI,PR,BC,BL,VC,units,item_nbr
0,48,33,41,16,37,39,24,0,716,1626,0.0,0.07,28.82,29.91,9.1,23,11.3,Los Angeles,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,1
1,48,33,41,16,37,39,24,0,716,1626,0.0,0.07,28.82,29.91,9.1,23,11.3,Los Angeles,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,2
2,48,33,41,16,37,39,24,0,716,1626,0.0,0.07,28.82,29.91,9.1,23,11.3,Los Angeles,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,3
3,48,33,41,16,37,39,24,0,716,1626,0.0,0.07,28.82,29.91,9.1,23,11.3,Los Angeles,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,4
4,48,33,41,16,37,39,24,0,716,1626,0.0,0.07,28.82,29.91,9.1,23,11.3,Los Angeles,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510483,45,33,39,-6,24,32,26,0,624,1646,0.1,0.02,29.51,30.24,20.4,34,20.9,San Jose,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,107
1510484,45,33,39,-6,24,32,26,0,624,1646,0.1,0.02,29.51,30.24,20.4,34,20.9,San Jose,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,108
1510485,45,33,39,-6,24,32,26,0,624,1646,0.1,0.02,29.51,30.24,20.4,34,20.9,San Jose,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,109
1510486,45,33,39,-6,24,32,26,0,624,1646,0.1,0.02,29.51,30.24,20.4,34,20.9,San Jose,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,110


In [7]:
weather_c.dtypes

tmax             int64
tmin             int64
tavg             int64
depart           int64
dewpoint         int64
wetbulb          int64
heat             int64
cool             int64
sunrise          int64
sunset           int64
snowfall       float64
preciptotal    float64
stnpressure    float64
sealevel       float64
resultspeed    float64
resultdir        int64
avgspeed       float64
city            object
+FC               bool
FC                bool
TS                bool
GR                bool
RA                bool
DZ                bool
SN                bool
SG                bool
GS                bool
PL                bool
IC                bool
FG+               bool
FG                bool
BR                bool
UP                bool
HZ                bool
FU                bool
VA                bool
DU                bool
DS                bool
PO                bool
SA                bool
SS                bool
PY                bool
SQ                bool
DR         